Template file from [https://github.com/squillero/computational-intelligence/blob/master/2024-25/set-cover.ipynb](https://github.com/squillero/computational-intelligence/blob/master/2024-25/set-cover.ipynb) 

# Set Cover problem

See: https://en.wikipedia.org/wiki/Set_cover_problem

In [1]:
from random import random, seed
import numpy as np

from icecream import ic

## Reproducible Initialization

If you want to get reproducible results, use `rng` (and restart the kernel); for non-reproducible ones, use `np.random`.

In [2]:
UNIVERSE_SIZE = 100_000
NUM_SETS = 10_000
DENSITY = 0.1

rng = np.random.Generator(np.random.PCG64([UNIVERSE_SIZE, NUM_SETS, int(10_000 * DENSITY)]))

In [3]:
# DON'T EDIT THESE LINES!

SETS = np.random.random((NUM_SETS, UNIVERSE_SIZE)) < DENSITY
for s in range(UNIVERSE_SIZE):
    if not np.any(SETS[:, s]):
        SETS[np.random.randint(NUM_SETS), s] = True
COSTS = np.pow(SETS.sum(axis=1), 1.1)

## Helper Functions

In [4]:
def valid(solution):
    """Checks wether solution is valid (ie. covers all universe)"""
    return np.all(np.logical_or.reduce(SETS[solution]))


def cost(solution):
    """Returns the cost of a solution (to be minimized)"""
    return COSTS[solution].sum()

## Have Fun!

In [5]:
# A dumb solution of "all" sets
solution = np.full(NUM_SETS, True)
ic(valid(solution), cost(solution))

ic| valid(solution): np.True_
    cost(solution): np.float64(251163170.0166925)


(np.True_, np.float64(251163170.0166925))

In [6]:
# A random solution with random 50% of the sets
solution = rng.random(NUM_SETS) < .5
ic(valid(solution), cost(solution))

ic| valid(solution): np.True_
    cost(solution): np.float64(124515264.92240825)


(np.True_, np.float64(124515264.92240825))

# Simple RHMC (Random Mutation Hill Climber)
source: lesson held on 03/10/2024

In [7]:
def single_mutation(solution):    #tweak - single mutation:
    new_sol = solution.copy()     #change a single value in a random position
    i = rng.integers(0, NUM_SETS) 
    new_sol[i] = not new_sol[i]  
    new_sol
    return new_sol

In [8]:
def multiple_mutation(solution):              #tweak - multiple mutation: 
    mask = rng.random(NUM_SETS) < .01         #create a boolean mask with 1% probability to be 'true'
    new_sol = np.logical_xor(solution, mask)  #xor: change the 'false' values to 'true' where the mask is 'true'
    return new_sol

In [9]:
def fitness(solution): #fitness function: tuple (lexicographic ordering)
    return (valid(solution), -cost(solution)) 

In [10]:
solution = rng.random(NUM_SETS) < .001      #random starting solution with low probability to have 'true' to start from an invalid solution
solution_fitness = fitness(solution)
first_solution_fitness = solution_fitness
ic(first_solution_fitness)
tweak = multiple_mutation                   #I choss the multiple mutation to tweak the solution

for steps in range(10_000):                      
    new_solution = tweak(solution)               #tweak the current solution
    new_solution_fitness = fitness(new_solution) #evaluate the new solution fitness
    if new_solution_fitness > solution_fitness:  #if the fitness of the new solution is better than the current one
        solution = new_solution                  #the current solution and its fitness are updated
        solution_fitness = fitness(solution)

ic(solution_fitness)

ic| first_solution_fitness: (np.False_, np.float64(-349719.1168954839))
ic| solution_fitness: (np.True_, np.float64(-2935504.62422972))


(np.True_, np.float64(-2935504.62422972))

# Greedy Algorithm
source: I used the algorithm from this website https://www.geeksforgeeks.org/greedy-approximate-algorithm-for-set-cover-problem/

In [11]:
def set_cover_greedy(SETS, COSTS):
    universe = set(range(SETS.shape[1])) #all elements
    covered = set()                      #covered elements
    selected = []                        #indexes of selected subsets, we start from an empty solution

    while covered!=universe:
        best_subset = None
        best_ratio = float("inf")

        for i in range(len(SETS)):
            subset = SETS[i]                            
            subset_elements = set(np.where(subset)[0]) #for each subset
            new_elements = subset_elements - covered   #I take the uncovered elements

            if new_elements:                  #if there are uncovered elements  
                current_cost = COSTS[i]       #I compute the ration cost/coverage
                coverage = len(new_elements)
                ratio = current_cost/coverage

                if ratio < best_ratio:        #if the ratio is the best to this moment
                    best_ratio = ratio        #I choose the current as best subset
                    best_subset = i

        if best_subset is not None:
            selected.append(best_subset)
            covered.update(set(np.where(SETS[best_subset])[0]))
        else:
            raise RuntimeError("No valid subset found. The problem may not be solvable with these sets and costs.")

    return selected

In [12]:
solution = set_cover_greedy(SETS=SETS, COSTS=COSTS)
ic(fitness(solution))

ic| fitness(solution): (np.True_, np.float64(-1528709.1532113561))


(np.True_, np.float64(-1528709.1532113561))